In [ ]:
import pandas as pd
import re

Read the .csv containing all Pokémon info into a DataFrame

In [ ]:
df = pd.read_csv('/content/pkm_info.csv')
df.head()

,name,evolution_info,types,Ability 1,Ability 2,Hidden Ability,HP,Atk,Def,Sp.Atk,Sp.Def,Spd,catch_rate,rarity,obtainability
0,Abra,It evolves into Kadabra as early as level 16; ...,['Psychic'],Synchronize,Inner Focus,Magic Guard,25,20,15,105,55,90,26.14%,Uncommon,"['Wild', 'NPCs']"
1,Aerodactyl,NaN,"['Rock', 'Flying']",Rock Head,Pressure,Unnerve,80,105,65,60,75,130,5.88%,Unhuntable,"['From diggable patches and Excavation Sites',..."
2,Alakazam,Having evolved from Kadabra when involved in a...,['Psychic'],Synchronize,Inner Focus,Magic Guard,55,50,45,135,85,120,6.54%,Unhuntable,[]
3,Arbok,"It evolves from Ekans as early as level 22, th...",['Poison'],Intimidate,Shed Skin,Unnerve,60,85,69,65,79,80,11.76%,Common,"['Wild', 'Discontinued Methods']"
4,Arcanine,It is the only evolutionary outlet of Growlith...,['Fire'],Intimidate,Flash Fire,Justified,90,110,80,100,80,95,9.8%,Uncommon,[]


Some attributes, such as *types* are in a list, let's split them into separate columns

In [ ]:
# Some attributes are in lists, let's split them into separate columns
for attribute in ['types']:
  # Remove [] and '
  df[attribute] = df[attribute].str.replace(r"[\[\]']", '', regex=True)
  if attribute != 'obtainability':
    # Find how many new columns will be created by each attribute split
    max_value = df[attribute].apply(lambda x: x.count(',') + 1).max()
    new_column_names = [f'{attribute}{x}' for x in range(1, max_value+1)]
    # Split into new columns
    df[new_column_names] = df[attribute].str.split(',', expand=True)
    # Drop original column
    df = df.drop(attribute, axis=1)

df.head()

,name,evolution_info,Ability 1,Ability 2,Hidden Ability,HP,Atk,Def,Sp.Atk,Sp.Def,Spd,catch_rate,rarity,obtainability,types1,types2
0,Abra,It evolves into Kadabra as early as level 16; ...,Synchronize,Inner Focus,Magic Guard,25,20,15,105,55,90,26.14%,Uncommon,"['Wild', 'NPCs']",Psychic,None
1,Aerodactyl,NaN,Rock Head,Pressure,Unnerve,80,105,65,60,75,130,5.88%,Unhuntable,"['From diggable patches and Excavation Sites',...",Rock,Flying
2,Alakazam,Having evolved from Kadabra when involved in a...,Synchronize,Inner Focus,Magic Guard,55,50,45,135,85,120,6.54%,Unhuntable,[],Psychic,None
3,Arbok,"It evolves from Ekans as early as level 22, th...",Intimidate,Shed Skin,Unnerve,60,85,69,65,79,80,11.76%,Common,"['Wild', 'Discontinued Methods']",Poison,None
4,Arcanine,It is the only evolutionary outlet of Growlith...,Intimidate,Flash Fire,Justified,90,110,80,100,80,95,9.8%,Uncommon,[],Fire,None


Further cleaning and normalization is needed:

In [ ]:
# Lowercase all column headers
df.columns = [x.lower() for x in df.columns]

# Replace . for _ if needed
df.columns = [re.sub('\.', '_', x) for x in df.columns]

# Correct typos
df = df.rename(columns={'types1': 'type1',
                        'types2': 'type2',
                        'ability 1': 'ability_1',
                        'ability 2': 'ability_2',
                        'hidden ability': 'hidden_ability'})

df.head()

,name,evolution_info,ability_1,ability_2,hidden_ability,hp,atk,def,sp_atk,sp_def,spd,catch_rate,rarity,obtainability,type1,type2
0,Abra,It evolves into Kadabra as early as level 16; ...,Synchronize,Inner Focus,Magic Guard,25,20,15,105,55,90,26.14%,Uncommon,"['Wild', 'NPCs']",Psychic,None
1,Aerodactyl,NaN,Rock Head,Pressure,Unnerve,80,105,65,60,75,130,5.88%,Unhuntable,"['From diggable patches and Excavation Sites',...",Rock,Flying
2,Alakazam,Having evolved from Kadabra when involved in a...,Synchronize,Inner Focus,Magic Guard,55,50,45,135,85,120,6.54%,Unhuntable,[],Psychic,None
3,Arbok,"It evolves from Ekans as early as level 22, th...",Intimidate,Shed Skin,Unnerve,60,85,69,65,79,80,11.76%,Common,"['Wild', 'Discontinued Methods']",Poison,None
4,Arcanine,It is the only evolutionary outlet of Growlith...,Intimidate,Flash Fire,Justified,90,110,80,100,80,95,9.8%,Uncommon,[],Fire,None


Now let's read the .csv containing the Pokémon list and their generation

In [ ]:
df_list = pd.read_csv('/content/pkm_list.csv')
df_list.head()

,name,wiki_url,gen
0,Abra,https://wiki.pokemonrevolution.net/index.php?t...,I
1,Aerodactyl,https://wiki.pokemonrevolution.net/index.php?t...,I
2,Alakazam,https://wiki.pokemonrevolution.net/index.php?t...,I
3,Arbok,https://wiki.pokemonrevolution.net/index.php?t...,I
4,Arcanine,https://wiki.pokemonrevolution.net/index.php?t...,I


We can join both DataFrames by using the *name* attribute as index

In [ ]:
df_merge = pd.concat([df.set_index('name'), df_list.set_index('name')], axis='columns')
df_merge.head()

,evolution_info,ability_1,ability_2,hidden_ability,hp,atk,def,sp_atk,sp_def,spd,catch_rate,rarity,obtainability,type1,type2,wiki_url,gen
name,,,,,,,,,,,,,,,,,
Abra,It evolves into Kadabra as early as level 16; ...,Synchronize,Inner Focus,Magic Guard,25,20,15,105,55,90,26.14%,Uncommon,"['Wild', 'NPCs']",Psychic,None,https://wiki.pokemonrevolution.net/index.php?t...,I
Aerodactyl,NaN,Rock Head,Pressure,Unnerve,80,105,65,60,75,130,5.88%,Unhuntable,"['From diggable patches and Excavation Sites',...",Rock,Flying,https://wiki.pokemonrevolution.net/index.php?t...,I
Alakazam,Having evolved from Kadabra when involved in a...,Synchronize,Inner Focus,Magic Guard,55,50,45,135,85,120,6.54%,Unhuntable,[],Psychic,None,https://wiki.pokemonrevolution.net/index.php?t...,I
Arbok,"It evolves from Ekans as early as level 22, th...",Intimidate,Shed Skin,Unnerve,60,85,69,65,79,80,11.76%,Common,"['Wild', 'Discontinued Methods']",Poison,None,https://wiki.pokemonrevolution.net/index.php?t...,I
Arcanine,It is the only evolutionary outlet of Growlith...,Intimidate,Flash Fire,Justified,90,110,80,100,80,95,9.8%,Uncommon,[],Fire,None,https://wiki.pokemonrevolution.net/index.php?t...,I


*catch_rate* is in string format, let's transform it into a float

In [ ]:
df_merge['catch_rate'] = df_merge['catch_rate'].str.replace('%', '').astype(float)

*gen* attribute is in roman numerals, we can exchange them for their alphanumeric counterpart

In [ ]:
# List every different gen entry
gen_list = (df_merge.groupby('gen')
                    .count()
                    .reset_index()['gen']
                    .to_list())

# For every roman numeral, exchange it for its alphanumeric counterpart
for index, gen in enumerate(gen_list):
  df_merge.loc[(df_merge['gen'] == gen), 'gen'] = index+1

We can check if the transformation above was successful

In [ ]:
df_merge.groupby('gen').count()

,evolution_info,ability_1,ability_2,hidden_ability,hp,atk,def,sp_atk,sp_def,spd,catch_rate,rarity,obtainability,type1,type2,wiki_url
gen,,,,,,,,,,,,,,,,
1,138,151,97,143,151,151,151,151,151,151,151,151,151,151,67,151
2,80,101,65,97,101,101,101,101,101,101,101,101,101,101,50,101
3,105,135,60,114,135,135,135,135,135,135,135,135,135,135,66,135
4,88,107,46,91,107,107,107,107,107,107,107,107,107,107,53,107
5,129,156,106,132,156,156,156,156,156,156,156,156,156,156,74,156
6,59,72,32,60,72,72,72,72,72,72,72,72,72,72,41,72
7,47,53,25,42,53,53,53,53,53,53,53,53,53,53,33,53


Aaaand export!

In [ ]:
df_merge.to_csv('pkm_info_clean.csv', index=False)